## Import Libraries

In [1]:
from os import path, makedirs
from itertools import product
import numpy as np
import scipy.io as sio

from brainpipe.system import study
from brainpipe.statistics import *

## Power Decoding - Partial//Detailed Encoding
### For ALL time points

In [2]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold as SKFold
from sklearn.model_selection import permutation_test_score
from sklearn.metrics import roc_auc_score
from numpy.random import permutation

st = study('Olfacto')
exp = 'Enc'#'Enc'
conds, subjects = ['low','high'],['CHAF','VACJ','SEMC','LEFC','PIRJ','FERJ']
# freqs = ['0_theta','1_alpha','2_beta','3_gamma']
freqs = ['delta','theta', 'alpha', 'beta','low_gamma','high_gamma']
path_pow = path.join(st.path, 'feature/TPSim_'+exp+'_By_Odor_By_Cond/TPS_by_cond/6freqs/')
#path_pow = path.join(st.path, 'feature/TPSim_Enc_Ret_By_Odor_all/TPS_R_p_by_cond/TPS_R_p_by_cond/')
filename = path.join(path_pow, 'TPS_spear_{}_cond_{}_{}_3s_zFisher.npz')
save_path = path.join(st.path, 'classified/TPSim_classif_'+exp[0]+'_by_cond_6freqs_3s_zFisher/')
#save_path = path.join(st.path, 'classified/Classif_TPSim_Enc_Ret_by_cond/')
nperm = 1000

for su in subjects:
    for freq in freqs:
        pow_list = []
        #=========================== Load Power files (nfreq, nelec, nwin, ntrial) =================================    
        mat0 = np.load(filename.format(su,conds[0],freq[:]),allow_pickle=True)
        pow_list.append(mat0['tps'])
        nelecs = mat0['tps'].shape[0]
        mat1 = np.load(filename.format(su,conds[1],freq[:]),allow_pickle=True)
        pow_list.append(mat1['tps'])
        print (su,mat0.files, 'TPS shape: ', [pow.shape for pow in pow_list])
        
        #=========================== Create dict for all results =================================    
        name_classif = (save_path+su +'_LDA_classif_'+freq+'_'+conds[0]+'_'+conds[1]+'_dissim.npz')
        if not path.exists(name_classif):
            kwargs = {}
            kwargs['names'], kwargs['channels'] = mat0['label'], mat0['channel']
            kwargs['xyz'] = mat0['xyz']

            # =========================== Select Power for 1 elec 1 freq =================================                 
            permut,auc = np.zeros((nelecs,nperm)),np.zeros((nelecs))
            for elec_num in range(nelecs):
                print('--» processing',su, 'elec', elec_num,'/',nelecs, 'freq',freq)
                pow_data_elec = [power[elec_num][:,np.newaxis] for power in pow_list]

                # create a data matrix, concatenate along the trial dimension
                x = np.concatenate(pow_data_elec, axis=0)
                print ('Size of the concatenated data: ', x.shape)
                y = np.hstack([np.array([i]*len(power)) for i, power in enumerate(pow_data_elec)])
                print ('Size of label for classif: ', len(y))

                clf = LDA()
                cv = SKFold(10)
                score, permutation_scores, pvalue = permutation_test_score(
                clf, x, y, scoring="roc_auc", cv=cv, n_permutations=nperm, n_jobs=-1)
                permut[elec_num] += permutation_scores
                auc[elec_num] += score
                print("Classification score %s (pvalue : %s)" % (score, pvalue))

            kwargs['perm'], kwargs['auc'] = permut, auc
            #Save plots
            if not path.exists(save_path):
                makedirs(save_path)
            np.savez(name_classif, **kwargs)
            del x, y, pow_data_elec, permutation_scores, score
        else:
            print(name_classif, 'already computed')
    del pow_list

-> Olfacto loaded
CHAF ['tps', 'pval', 'label', 'channel', 'xyz'] TPS shape:  [(57, 15), (57, 105)]
--» processing CHAF elec 0 / 57 freq delta
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.5890909090909091 (pvalue : 0.3086913086913087)
--» processing CHAF elec 1 / 57 freq delta
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.5290909090909091 (pvalue : 0.5464535464535465)
--» processing CHAF elec 2 / 57 freq delta
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.5872727272727273 (pvalue : 0.3386613386613387)
--» processing CHAF elec 3 / 57 freq delta
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.4663636363636364 (pvalue : 0.6443556443556444)
--» processing CHAF elec 4 / 57 freq delta
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.7254545454545455 

Classification score 0.5981818181818183 (pvalue : 0.2547452547452547)
--» processing CHAF elec 44 / 57 freq delta
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.8527272727272728 (pvalue : 0.000999000999000999)
--» processing CHAF elec 45 / 57 freq delta
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.26 (pvalue : 0.954045954045954)
--» processing CHAF elec 46 / 57 freq delta
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.8345454545454546 (pvalue : 0.000999000999000999)
--» processing CHAF elec 47 / 57 freq delta
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.6318181818181817 (pvalue : 0.13586413586413587)
--» processing CHAF elec 48 / 57 freq delta
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.5427272727272727 (pvalue : 0.5164835164835165)
--» p

Classification score 0.4118181818181818 (pvalue : 0.7152847152847153)
--» processing CHAF elec 31 / 57 freq theta
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.8209090909090909 (pvalue : 0.000999000999000999)
--» processing CHAF elec 32 / 57 freq theta
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.37545454545454543 (pvalue : 0.7932067932067932)
--» processing CHAF elec 33 / 57 freq theta
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.19181818181818183 (pvalue : 0.9920079920079921)
--» processing CHAF elec 34 / 57 freq theta
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.28454545454545455 (pvalue : 0.9230769230769231)
--» processing CHAF elec 35 / 57 freq theta
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.4818181818181818 (pvalue : 0.64235764

Classification score 0.23727272727272725 (pvalue : 0.9820179820179821)
--» processing CHAF elec 18 / 57 freq alpha
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.5472727272727272 (pvalue : 0.48151848151848153)
--» processing CHAF elec 19 / 57 freq alpha
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.5790909090909091 (pvalue : 0.3676323676323676)
--» processing CHAF elec 20 / 57 freq alpha
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.649090909090909 (pvalue : 0.14185814185814186)
--» processing CHAF elec 21 / 57 freq alpha
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.6 (pvalue : 0.2597402597402597)
--» processing CHAF elec 22 / 57 freq alpha
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.7172727272727274 (pvalue : 0.01098901098901099)
--» pro

Classification score 0.6672727272727272 (pvalue : 0.061938061938061936)
--» processing CHAF elec 5 / 57 freq beta
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.2290909090909091 (pvalue : 0.981018981018981)
--» processing CHAF elec 6 / 57 freq beta
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.6172727272727273 (pvalue : 0.2047952047952048)
--» processing CHAF elec 7 / 57 freq beta
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.4518181818181818 (pvalue : 0.6873126873126874)
--» processing CHAF elec 8 / 57 freq beta
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.519090909090909 (pvalue : 0.5594405594405595)
--» processing CHAF elec 9 / 57 freq beta
Size of the concatenated data:  (120, 1)
Size of label for classif:  120
Classification score 0.59 (pvalue : 0.3126873126873127)
--» processing CHAF

...

Classification score 0.4328388278388278 (pvalue : 0.7982017982017982)
--» processing FERJ elec 41 / 45 freq theta
Size of the concatenated data:  (227, 1)
Size of label for classif:  227
Classification score 0.5608669108669109 (pvalue : 0.12987012987012986)
--» processing FERJ elec 42 / 45 freq theta
Size of the concatenated data:  (227, 1)
Size of label for classif:  227
Classification score 0.5917521367521367 (pvalue : 0.030969030969030968)
--» processing FERJ elec 43 / 45 freq theta
Size of the concatenated data:  (227, 1)
Size of label for classif:  227
Classification score 0.5987545787545788 (pvalue : 0.025974025974025976)
--» processing FERJ elec 44 / 45 freq theta
Size of the concatenated data:  (227, 1)
Size of label for classif:  227
Classification score 0.5495787545787546 (pvalue : 0.1978021978021978)
FERJ ['tps', 'pval', 'label', 'channel', 'xyz'] TPS shape:  [(45, 136), (45, 91)]
--» processing FERJ elec 0 / 45 freq alpha
Size of the concatenated data:  (227, 1)
Size of lab

Classification score 0.682997557997558 (pvalue : 0.000999000999000999)
--» processing FERJ elec 40 / 45 freq alpha
Size of the concatenated data:  (227, 1)
Size of label for classif:  227
Classification score 0.6977655677655676 (pvalue : 0.000999000999000999)
--» processing FERJ elec 41 / 45 freq alpha
Size of the concatenated data:  (227, 1)
Size of label for classif:  227
Classification score 0.5525579975579975 (pvalue : 0.18281718281718282)
--» processing FERJ elec 42 / 45 freq alpha
Size of the concatenated data:  (227, 1)
Size of label for classif:  227
Classification score 0.6367155067155067 (pvalue : 0.000999000999000999)
--» processing FERJ elec 43 / 45 freq alpha
Size of the concatenated data:  (227, 1)
Size of label for classif:  227
Classification score 0.6200427350427351 (pvalue : 0.004995004995004995)
--» processing FERJ elec 44 / 45 freq alpha
Size of the concatenated data:  (227, 1)
Size of label for classif:  227
Classification score 0.36081196581196584 (pvalue : 0.9830

Classification score 0.6196947496947496 (pvalue : 0.004995004995004995)
--» processing FERJ elec 39 / 45 freq beta
Size of the concatenated data:  (227, 1)
Size of label for classif:  227
Classification score 0.5528998778998779 (pvalue : 0.17382617382617382)
--» processing FERJ elec 40 / 45 freq beta
Size of the concatenated data:  (227, 1)
Size of label for classif:  227
Classification score 0.561923076923077 (pvalue : 0.12387612387612387)
--» processing FERJ elec 41 / 45 freq beta
Size of the concatenated data:  (227, 1)
Size of label for classif:  227
Classification score 0.5089133089133089 (pvalue : 0.5754245754245755)
--» processing FERJ elec 42 / 45 freq beta
Size of the concatenated data:  (227, 1)
Size of label for classif:  227
Classification score 0.4640842490842491 (pvalue : 0.6693306693306693)
--» processing FERJ elec 43 / 45 freq beta
Size of the concatenated data:  (227, 1)
Size of label for classif:  227
Classification score 0.5998107448107447 (pvalue : 0.016983016983016

Classification score 0.6139682539682539 (pvalue : 0.006993006993006993)
--» processing FERJ elec 37 / 45 freq low_gamma
Size of the concatenated data:  (227, 1)
Size of label for classif:  227
Classification score 0.3652869352869353 (pvalue : 0.975024975024975)
--» processing FERJ elec 38 / 45 freq low_gamma
Size of the concatenated data:  (227, 1)
Size of label for classif:  227
Classification score 0.5231868131868131 (pvalue : 0.4755244755244755)
--» processing FERJ elec 39 / 45 freq low_gamma
Size of the concatenated data:  (227, 1)
Size of label for classif:  227
Classification score 0.7847802197802198 (pvalue : 0.000999000999000999)
--» processing FERJ elec 40 / 45 freq low_gamma
Size of the concatenated data:  (227, 1)
Size of label for classif:  227
Classification score 0.7816117216117215 (pvalue : 0.000999000999000999)
--» processing FERJ elec 41 / 45 freq low_gamma
Size of the concatenated data:  (227, 1)
Size of label for classif:  227
Classification score 0.6125824175824175 

Classification score 0.4195665445665445 (pvalue : 0.8691308691308691)
--» processing FERJ elec 35 / 45 freq high_gamma
Size of the concatenated data:  (227, 1)
Size of label for classif:  227
Classification score 0.4295909645909646 (pvalue : 0.8481518481518482)
--» processing FERJ elec 36 / 45 freq high_gamma
Size of the concatenated data:  (227, 1)
Size of label for classif:  227
Classification score 0.7614957264957265 (pvalue : 0.000999000999000999)
--» processing FERJ elec 37 / 45 freq high_gamma
Size of the concatenated data:  (227, 1)
Size of label for classif:  227
Classification score 0.4997619047619047 (pvalue : 0.5734265734265734)
--» processing FERJ elec 38 / 45 freq high_gamma
Size of the concatenated data:  (227, 1)
Size of label for classif:  227
Classification score 0.5453418803418802 (pvalue : 0.2517482517482518)
--» processing FERJ elec 39 / 45 freq high_gamma
Size of the concatenated data:  (227, 1)
Size of label for classif:  227
Classification score 0.715158730158730